## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Содержание работы

<ul style="list-style:none; text-decoration: none">
    <li><a style="text-decoration: none" href="#introduction">Шаг 1. Откройте файл с данными и изучите общую информацию.</a></li>
    <li><a style="text-decoration: none" href="#data_pre-processing">Шаг 2. Предобработка данных</a></li>
    <ul>
        <li><a style="text-decoration: none" href="#missing_data-treatment">Обработка пропусков</a></li>
        <li><a style="text-decoration: none" href="#replacing_data-type">Замена типа данных</a></li>
        <li><a style="text-decoration: none" href="#processing_duplicates">Обработка дубликатов</a></li>
        <li><a style="text-decoration: none" href="#lemmatization">Лемматизация</a></li>
        <li><a style="text-decoration: none" href="#categorization">Категоризация данных</a></li>
    </ul> 
    <li><a style="text-decoration: none" href="#questions">Шаг 3. Ответьте на вопросы</a></li>
    <li><a style="text-decoration: none" href="#conclusion">Шаг 4. Общий вывод</a></li>
</ul>

## Откройте файл с данными и изучите общую информацию. <a name="introduction"></a>

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem() 

Читаем файл с помощью команды read_csv

In [2]:
credit_data = pd.read_csv("/datasets/data.csv")

Изучаем общую информацию о предоставленном нами данных

In [3]:

credit_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Просмотрим некоторых основных статистических данные, таких как процентиль, среднее значение, стандартное отклонение в таблице

In [4]:
credit_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

Всего в таблице 12 столбцов, из которых 7 относятся к количественным данным, а 5 к категориальным.
Каждая строка таблицы содержит информацию о заемщике, а именно информацию о его семейном положении, количестве детей, образовании, трудовом стаже, его доходе, наличии или отсутствии задолженности по возврату кредитов и цели взятия кредита. При этом в таблице существует два показателя, у которых есть идентификатор - уровень образование и семейное положение. Количество значений в столбцах total_income и days_employed отличается от других столбцев. Это говорит о том, что в данных есть пропущенные значения.
Три проблемы, которые нужно решить: пропуски, отрицательное значение у столбца dob_years и children, неправильные значения в столбце income_type. Для проверки рабочих гипотез особенно ценны столбцы children, education, total_income и debt.

##  Предобработка данных  <a name="data_pre-processing"></a>

### Обработка пропусков <a name="missing_data-treatment"></a>

Просматриваем пропуски в таблице

In [5]:
credit_data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пишем функцию, которая ищет медиану среди заданной категории в столбце и дальше производим замену пропущенных значений  на полученное значение в таблице

In [6]:
def missing_data(data, array, column):
    for row in array:
        data_value = data[credit_data['income_type'] == row][column].median()
        data.loc[data['income_type'] == row, column] =  data[data['income_type'] == row][column].fillna(data_value)

создаем массив для значений столбца 'income_type' для использования ее в функции

In [7]:
value_missing = credit_data['income_type'].unique()

Применяем функцию меняющую пропущенные значения в столбце "total_income" на медиану значений по столбцу "income_type" 

In [8]:

missing_data(credit_data, value_missing, "total_income")

In [9]:
credit_data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Начинаем работу со столбцом 'days_employed', где имеются отрицательные значения и некоторые данные отражены в часах, а не в количестве дней.
Переводим отрицательные значения в положительные в столбце'days_employed'

In [10]:
credit_data['days_employed'] = credit_data['days_employed'].abs()

Переводим некоторые значения из часов в количество дней, при этом мы допустили, что человек не может работать в своей жизни больше чем 18 000 дней, на основе этого была проведена выборка значений, которые нужно поменять 

In [11]:
credit_data.loc[credit_data['days_employed'] >= 18000, 'days_employed'] = credit_data['days_employed']/30

Для работающих граждан пропущенные значения были заменены на  возможное количество трудового стажа, при условии, что человек начал работать с 19 лет

In [12]:
credit_data.loc[(credit_data['days_employed'].isnull()) & (credit_data['income_type'] != "пенсионер"), 'days_employed']=(credit_data["dob_years"]-18)*365

Для песионеров пропущенные значения были заменены на полученную медиану трудового стажа пенсионеров

In [13]:
array = ["пенсионер"] 
missing_data(credit_data, array, 'days_employed')

In [14]:
credit_data.loc[credit_data['days_employed'] == -6570, 'days_employed'] = credit_data[credit_data['days_employed'] == -6570]['days_employed'].dropna().reset_index(drop
= True)

In [15]:
credit_data = credit_data.dropna().reset_index(drop= True)

In [16]:
credit_data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Проверяем другие столбцы, в столбце "dob_years" есть нулевые значения, что невозможно само по себе, находим их количество

In [17]:
credit_data[credit_data["dob_years"] == 0]["dob_years"].count()

94

Заменяем нулевые значения на предполагаемый их возраст при условии, что они начали работать с 19 лет

In [18]:
credit_data.loc[credit_data["dob_years"] == 0, "dob_years"] = (credit_data["days_employed"]/365) + 18

Проверка пропущенных значений

In [19]:
credit_data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Проверка значений столбца "children", выявлено отклонение в наличии значения 20

In [20]:
credit_data["children"].value_counts()

 0     14144
 1      4817
 2      2055
 3       329
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [21]:
credit_data.loc[(credit_data["children"] == 20) & (credit_data["family_status"] != "Не женат / не замужем"),"children"] = 2
credit_data.loc[(credit_data["children"] == 20) & (credit_data["family_status"] == "Не женат / не замужем"),"children"] = 0
credit_data.loc[credit_data["children"] == -1,"children"] = 1


Проверяем полученную замену

In [22]:
credit_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,11342.202402,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [23]:
credit_data["children"].value_counts()

0    14153
1     4864
2     2122
3      329
4       41
5        9
Name: children, dtype: int64

Проверяем, чтобы в столбце days_employed не было отрицательных значений, а в столбце dob_years нулевые значения

In [24]:
credit_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21518.000000,21518.000000,21518.000000,21518.000000,21518.000000,21518.000000,2.151800e+04
mean,0.478855,4584.069935,43.437634,0.817223,0.972581,0.080909,1.652299e+05
std,0.754835,4495.084954,12.280540,0.548191,1.420343,0.272702,9.805898e+04
min,0.000000,24.141633,18.298540,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1024.854208,33.000000,1.000000,0.000000,0.000000,1.077824e+05
50%,0.000000,2559.046304,43.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,7810.618031,53.000000,1.000000,1.000000,0.000000,1.956010e+05
max,5.000000,19710.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

Как предполагалось ранее пропуски были найдены в двух столбцах total_income и days_employed, при этом была выявлена их взаимосвязь друг с другом, то есть они пропущены в одни строчках. Первоначальная гипотеза о том, что это быть может безработные не подтвердилась, так как большая часть из них подрадают в категорию "сотрудники" - 50,8%, затем идут категория компаньоны 23,3%, потом "пенсионеры" - 18,9%, а замыкают список госслужащие (6,7%) и один предприниматель. 
Поэтому скорее всего пропуски можно отнести к сбоям в системе, или человеческому фактору, также это относится к найденным ошибкам в столбцах children и dob_years, предположу, что это скорее всего отпечатки.   

В столбце days_employed были убраны отрицательные значения. Что касается пропущенных значений, то мною было предположено, что человек в среднем начинает работать с 19 лет, поэтому пропущенные значения для работающих граждан были заменены на возможное количество трудового стажа, которое расчитано было как разница между возрастом заемщика и числом  18, а затем мною был совершен перевод полученного значения в часы с помощью умножения на количество дней в году - 365. Так как пенсионеры уже не работают, такие значения подставлять было некорректно, поэтому я с помощью написанной функции рассчитал медиану по трудовому стажу всех пенсионеров и вставил полученные значения вместо пропущенных.

В столбце total_income пропущенные значения были заменены на медиану общих доходов сгруппированнных на основе типов работника.

В столбце dob_years найдены нулевые значения и была произведена замена их на предполагаемый  возраст, который был найден с помощью деления трудового стажа на количество дней в годы и прибавления числа 18, то есть я предположил, что человек начал работать с 19 лет.

В столбце children выявлено значение 20, которое было заменено для людей, которые в браке / разведены / вдовцы, на 2, а для людей, которые не женаты, на 0, так как вероятность у таких людей иметь 2 детей очень маловероятно.


### Замена типа данных <a name="replacing_data-type"></a>

Заменим вещественный тип данных на целочисленный в строчках dob_years, days_employe и total_income

In [25]:
credit_data["dob_years"] = credit_data["dob_years"].astype("int")
credit_data["days_employed"] = credit_data["days_employed"].astype("int")
credit_data["total_income"] = credit_data["total_income"].astype("int")

### Вывод

Для удобства чтения таблицы были заменены вещественные типы данных на целочисленные в строчках dob_years, days_employe и total_income. Для этого использовалась функция astype("int")

### Обработка дубликатов <a name="processing_duplicates"></a>


Dыявляем по уникальным значениям, что дубликаты содержатся в столбце "education" из-за отличия регистра нескольких букв 

In [26]:
credit_data["education"].value_counts()

среднее                13744
высшее                  4717
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Приводим все к нижнему регистру

In [27]:
credit_data["education"] = credit_data["education"].str.lower() 

Проверяем, что дубликаты все ушли

In [28]:
credit_data["education"].value_counts()

среднее                15227
высшее                  5259
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

При проверки столбца family_status одно значение было написано с большой буквы

In [29]:
credit_data["family_status"].value_counts()

женат / замужем          12376
гражданский брак          4175
Не женат / не замужем     2812
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Приводим все к единому стилю, чтобы в дальнейшем избежать ошибок

In [30]:
credit_data["family_status"] = credit_data["family_status"].str.lower() 

### Вывод

Были выявлены с помощью функции value_counts() дубликаты в столбце "education" из-за отличия регистра нескольких букв, данная ошибка была исправлена с помощью функции .str.lower(), которая привела все к нижнему регистру.
Для красоты в столбце family_status привел все к единому стилю, так как значение "Не женат / не замужем" начиналось с большой буквы

### Лемматизация <a name="lemmatization"></a>

Создадим функцию, которая лемматизирует значения из заданного столбца таблицы 

In [31]:
def purpose_new(purpose):
    lemmatize = "".join(m.lemmatize(purpose))
    return lemmatize

Применение созданной функции к столбцу purpose с помощью функции apply(), на основе чего создается новый столбец purpose_new с леммами

In [32]:
credit_data['purpose_new'] = credit_data['purpose'].apply(purpose_new)

Проверяем работу функции

In [33]:
credit_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование\n
4,0,11342,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n


### Вывод

Для дальнейшей категоризации данных, чтобы можно было бы выделить по столбцу "purpose" более редкие значения был применен метод лемматизации, который приводит слова к его словарной форме (лемме). Для этого была создана функция которая лемматизирует значения из заданного столбца таблицы и затем с помощью функции apply() данную функцию применил к столбцу purpose. В таблице образовалась новый столбец  purpose_new с леммами. 


### Категоризация данных <a name="categorization"></a>

Сделал функцию, которая категоризирует цели на четыре большие группы "автомобиль", "свадьба", "образование" и "жилье" на основе полученных леммов


In [34]:
def purpose_group(purpose):
    if "автомобиль" in purpose:
        return "автомобиль"
    elif "свадьба" in purpose:
        return "свадьба"
    elif "образование" in purpose:
         return "образование"
    elif "жилье" in purpose or "недвижимость" in purpose:
         return "жилье"
    return 'Нет категории' 

Применяем функцию к столбцу purpose_new с леммами

In [35]:
credit_data['purpose_target'] = credit_data['purpose_new'].apply(purpose_group)

Сделал функцию, которая категоризирует количество детей на следующие группы "нет детей", "есть один ребенок" и "больше одного ребенка"  

In [36]:
def presence_children(children):
    if children == 0:
         return "нет детей"
    elif children == 1:
        return "есть один ребенок"
    return "больше одного ребенка"

Применяем функцию к столбцу children с леммами

In [37]:
credit_data['presence_children'] = credit_data['children'].apply(presence_children)

Создаем словарь с тремя группами: "высокий достаток", "чуть выше среднего", "средний достаток", в котором прописал значение, с которого начинается даннная категория.
Делаем функцию, которая категоризирует заемщиков на основе их дохода, при этом используется словарь для отнесения каждого заемщика к конкретной категории.


In [38]:
credit_data['total_income'].median()

142594.0

In [39]:
credit_data.loc[credit_data['total_income']> 143000, 'total_income'].mean()

227633.71351555848

In [40]:
credit_data.loc[credit_data['total_income']< 143000, 'total_income'].mean()

104946.114938328

In [41]:
d = dict([("высокий достаток", 228000), ("выше среднего заработка", 143000), ("ниже среднего заработка", 104000)])

def category_creditors(income):
    if income > d["высокий достаток"]:
        return "люди с высоким достатком"
    elif income > d["выше среднего заработка"]:
        return "люди с чуть выше средним достатком"
    elif income > d["ниже среднего заработка"]:
        return "люди с чуть ниже средним достатком"
    return "люди с низким достатком"

Применил функцию к столбцу category_creditors

In [42]:
credit_data['category_creditors'] = credit_data['total_income'].apply(category_creditors)

In [43]:
#проверил на уникальные значения
credit_data['category_creditors'].value_counts()

люди с чуть выше средним достатком    6950
люди с чуть ниже средним достатком    5992
люди с низким достатком               4953
люди с высоким достатком              3623
Name: category_creditors, dtype: int64

### Вывод

Исходя из поставленных передо мной вопросов я решил сделать три классификации:
1. классификация по целям на следующие группы - <b>"автомобиль", "свадьба", "образование" и "жилье"</b> на основе;
2. классификация по количеству детей на следующие группы - <b>"нет детей", "есть один ребенок" и "больше одного"</b>;
3. классификация по уровню доходов заемщика на следующие группы - <b>"высокий достаток" (от 228000 рублей), "чуть выше среднего" (от 143000 рублей), "чуть ниже среднего" (от 104000 рублей) и "люди с низким достатком" (ниже 104000 рублей)</b>. Данные были получены на основе полученной медианы по доходам данных. На основе полученной медианы находим среднее значение по людям, у которых доходы выше среднего и ниже среднего.

Для каждой классификации была создана функция, на вход которой попадает определенное значение, а возвращает она категорию заемщика. При этом в случае классификации по уровню доходов заемщика был создан словарь, так как в качестве отсева использовались числа. Поэтому было уместно создать словарь.
Затем в каждом случае использовал функцию apply() к конкретномe столбцу, которая брала значение столбца датафрейма и применяла к нему функцию из своего аргумента. Таким образом в таблице появились три новых столбца - category_creditors, presence_children и purpose_target. 
   




## Ответьте на вопросы <a name="questions"></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Группировка по наличию детей без  разделения группы "больше одного ребенка" с использованием функции agg(), которая применяет к столбцу "debt"  функцию sum(), count() 

In [44]:
children_table = credit_data.groupby(["presence_children"]).agg({'debt':['sum', 'count']}).reset_index()

Создание нового столбца, который рассчитывает процент невозврата по кредиту.
Проверка таблицы.

In [45]:
children_table['percentages_non-repayment'] = children_table['debt']['sum'] / children_table['debt']['count'] * 100
children_table

presence_children  debt        percentages_non-repayment
                           sum  count                          
0  больше одного ребенка   232   2501                  9.276289
1      есть один ребенок   445   4864                  9.148849
2              нет детей  1064  14153                  7.517841

Группировка по наличию детей с разделением по количеству детей группы "больше одного ребенка" с использованием функции agg(), которая применяет к столбцу "debt"  функцию sum(), count() 

In [46]:
children_table1 = credit_data.groupby(["presence_children", "children"]).agg({'debt':['sum', 'count']}).reset_index()



Создание нового столбца, который рассчитывает процент невозврата по кредиту
Проверка таблицы.

In [47]:
children_table1['percentages_non-repayment'] = children_table1['debt']['sum'] / children_table1['debt']['count'] * 100
children_table

presence_children  debt        percentages_non-repayment
                           sum  count                          
0  больше одного ребенка   232   2501                  9.276289
1      есть один ребенок   445   4864                  9.148849
2              нет детей  1064  14153                  7.517841

### Вывод

В целом зависимость подтвердилась, так как согласно полученным данным, чем больше детей, тем больше процент невозврата кредита в срок, так данный показатель по группе "больше одного ребенка" составил 9.3%, на втором месте с одним ребенком (9,14%). При этом при рассмотрении более детально группы "больше одного ребенка", наибольший процент невозврата кредита в срок у заемщиков, у которых четыре ребенка - 9,75% и у которых два ребенка -9.47%. Но здесь необходимо понимать, что  не так много людей у которых 4 ребенка, следовательно таких заемщиков очень немного. Самые дисциплинированные по выплате кредита являются заемщики, которые не имеют детей, по ним самый низкий процент невозврата - 7,5%.

При этом выбор метода agg для выявления зависимостей неслучаен, так как он позволяет создать дополнительный стоблец по проценту невозврата. При этом можно было бы использовать функцию для нахождения процента mean(), но так как в любом случае пришлось для удобства умножать на 100 полученное значение, мною было решено найти процент невозврата как отношение количества заемщиков, по которым есть задолженность по кредиту, к общему количеству заещиков с разделением их на группы. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Группировка по семейному положению с использованием функции agg(), которая применяет к столбцу "debt"  функцию sum(), count()

In [48]:
family_table = credit_data.groupby(["family_status"]).agg({'debt':['sum', 'count']}).reset_index()


Создание нового столбца, который рассчитывает процент невозврата по кредиту

In [49]:
family_table['percentages_non-repayment'] = family_table['debt']['sum'] / family_table['debt']['count'] * 100


#вызов таблицы с сортировкой по столбцу percentages_non-repayment с убыванием

In [50]:
family_table.sort_values(by="percentages_non-repayment", ascending=False)

family_status debt        percentages_non-repayment
                          sum  count                          
4  не женат / не замужем  274   2812                  9.743954
2       гражданский брак  388   4175                  9.293413
3        женат / замужем  931  12376                  7.522624
0              в разводе   85   1195                  7.112971
1         вдовец / вдова   63    960                  6.562500

### Вывод

В целом зависимость не подтвердилась, так как больше всего не уплачивают кредит в срок заемщики, которые являются холостыми (9,7%) или те кто в гражданском браке (9,3%), при этом меньше всего посроченности по платежам наблюдается по заемщикам, которые в разводе (7,1%) или являются вдовцами (6,5%). То есть нет четкой зависимости между тем, что человек является холостым / разведенным / вдовцом (то есть у него нет пары) или он/ она находится в официальном браке / гражданском браке. Хотя если посмотреть цифры, то больше всего кредита берут заемщики, находящиеся в браке, и в целом у них нормальный показатель по возврату в срок кредита - (7,5%). То есть данная группа является более благонадежной по сравнению с людьми, которые являются холостыми или которые в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Группировка по уровню дохода с использованием функции agg(), которая применяет к столбцу "debt"  функцию sum(), count()

In [51]:
income_table = credit_data.groupby(["category_creditors"]).agg({'debt':['sum', 'count']}).reset_index()

Создание нового столбца, который рассчитывает процент невозврата по кредиту

In [52]:
income_table['percentages_non-repayment'] = income_table['debt']['sum'] / income_table['debt']['count'] * 100

Вызов таблицы с сортировкой по столбцу percentages_non-repayment с убыванием

In [53]:
income_table.sort_values(by="percentages_non-repayment", ascending=False)

category_creditors debt       percentages_non-repayment
                                       sum count                          
3  люди с чуть ниже средним достатком  526  5992                  8.778371
2  люди с чуть выше средним достатком  568  6950                  8.172662
1             люди с низким достатком  390  4953                  7.874016
0            люди с высоким достатком  257  3623                  7.093569

### Вывод

- Как разные цели кредита влияют на его возврат в срок?

Группировка по целям кредита с использованием функции agg(), которая применяет к столбцу "debt"  функцию sum(), count()

In [54]:
purpose_table = credit_data.groupby(["purpose_target"]).agg({'debt':['sum', 'count']}).reset_index()

Создание нового столбца, который рассчитывает процент невозврата по кредиту


In [55]:
purpose_table['percentages_non-repayment'] = purpose_table['debt']['sum'] / purpose_table['debt']['count'] * 100

Вызов таблицы с сортировкой по столбцу percentages_non-repayment с убыванием

In [56]:
purpose_table.sort_values(by="percentages_non-repayment", ascending=False)

purpose_target debt        percentages_non-repayment
                  sum  count                          
0     автомобиль  403   4315                  9.339513
2    образование  370   4022                  9.199403
3        свадьба  186   2347                  7.925011
1          жилье  782  10834                  7.218017

### Вывод

На удивление лучше всего возвращают кредит в срок заемщики, которые взяли ипотеку, хотя это самый "дорогой" вид кредита, так как стоимость квартиры отличается от стоимости автомобиля, образования и свадьбы. Хотя это можно объяснить тем, что штрафы по данному кредиту больней всего могут ударить по заемщику и в целом пока заемщик не выплатит в полном объеме ипотеку, квартира находится в залоге у банка.
По кредиту на свадьбу маленький процент невозврата в срок (7,9%). Это можно объяснить тем, что сумма по данному виду кредита является небольшой, и в целом вдвоем его легко погасить. 
А вот самые "ветреные" заемщики это люди, которые берут в автокредит (процент невозврата  - 9,3%) и кредит на образование (процент невозврата - 9,2%). Скорее всего в данной группе невозвращают кредит молодые люди, у которых нет стабильного дохода или доход находится пока не на очень высоком уровне, но хочется иметь машину или получить образование.

### Шаг 4. Общий вывод  3

На основе полученных данных от заказчика в лице кредитный отдел банка была рассмотрена взаимосвязь семейного положения, количество детей, уровень дохода и целей кредита на факт погашения кредита в срок. Данные включали в себя информацию о его семейном положении, количестве детей, образовании, трудовом стаже, его доходе, наличии или отсутствии задолженности по возврату кредитов и цели взятия кредита. 

После предобработки полученных данных, мною было решено сделать категоризацию по следующим группам: по наличию детей у заемщика (ни одного, один ребенок или больше одного ребенка, по целям (автомобиль, свадьба, образование и жилье) и по уровню доходов заемщика (высокий достаток, чуть выше среднего, чуть ниже среднего и низкий достаток). Данное разделение на группы позволило провести комплексный анализ. 

В рамках выполненной работы были получены следующие выводы:
 - взаимосвязь количество детей на факт погашения кредита в срок подтвердилась. Заемщики у которых более одного ребенка хуже всего возвращают кредит (процент невозврата составил 9.3%), при этом если детально рассмотреть данную группу, то хуже всего возвращают заемщики с четверьмя детьми (9,75%)и с двумя детьми. У заемщиков с одним ребенком процент невозврата тоже высокий (9,14%). Лучше всего возвращают кредит заемщики, у которых нет ребенка (процент невозврата составил 7.5%).  
 - взаимосвязь семейного положения на факт погашения кредита в срок не подтвердилась. Больше всего не уплачивают кредит в срок заемщики, которые являются холостыми (9,7%) или те кто в гражданском браке (9,3%), при этом меньше всего посроченности по платежам наблюдается по заемщикам, которые в разводе (7,1%) или являются вдовцами (6,5%). То есть нет четкой зависимости между тем, что человек является холостым / разведенным / вдовцом (то есть у него нет пары) или он/ она находится в официальном браке / гражданском браке.
 - взаимосвязь уровня дохода на факт погашения кредита в срок не подтвердилась. Люди с высоким достатком лучше всего платят по кредиту (процент невозврата - 7,09%), но в тоже время люди с низким достатком тоже относятся к благонадежным плательщикам (7,9%), что скорее всего обусловлено тем, что им дают небольшой кредит, который они в состоянии покрыть. Хуже всего платят в срок заемщики с чуть ниже средним достатком (8,85).
 - лучше всего возвращают кредит в срок заемщики, которые взяли ипотеку (процент невозврата 7,2%) или кредит на свадьбу (7,9%). Скорее всего это связано с тем, что люди осознанно уже берут кредит на покупку недвижимости или на свадьбу. А меньше всего возвращают в срок автокредит (9,3%) и кредит на образование (9,2%). Можно предположить, что больше всего в данных группах невозвращают кредит молодые люди, у которых нет стабильного дохода или доход находится пока не на очень высоком уровне, но хочется иметь машину или получить образование.
 
Исходя из проанализированных данных можно составить портрет идеального заемщика:
- без ребенка или с одним ребенком;
- находится в официальном браке;
- обладает высоким достатком;
- цель кредита - покупка недвижимости.